In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import utils
from multiprocessing import cpu_count, Pool

num_cores = cpu_count() #number of cores on your machine
num_partitions = num_cores #number of partitions to split dataframe

%matplotlib inline

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sns.set_style('whitegrid')

plt.rcParams['figure.figsize'] = (16.0, 8.0)

In [18]:
carac = pd.read_csv("dataset/caracteristics.csv", encoding="latin-1")
holi = pd.read_csv("dataset/holidays.csv", encoding="utf-8")

/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def parallelize_df(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [20]:
def to_datetime(r):
    y = r['an']
    m = r['mois']
    d = r['jour']
    hhmm = "%04d" % (r['hrmn'])
    hh = hhmm[:2]
    mm = hhmm[-2:]
    date = "2%03d/%02d/%02d %s:%s" % (y , m, d, hh, mm)
    r = pd.to_datetime(date, format='%Y/%m/%d %H:%M')
    return r

def par(data):
    return data.apply(to_datetime, axis=1)
 
carac['datetime'] = utils.parall_df(carac, par)

In [22]:
carac['date'] = carac['datetime'].dt.date
# Monday=0, Sunday=6
carac['weekdays'] = carac['datetime'].dt.dayofweek

In [28]:
from datetime import timedelta

holidays = pd.to_datetime(holi['ds'])
delta = timedelta(days=7)

def holiday_dist(df):
    date = df['date']
    from_date = date - delta
    to_date = date + delta
    mask = (holidays >= from_date) & (holidays <= to_date)
    dist_day = date - holidays[mask]
    return dist_day

def par(df):
    return df.apply(holiday_dist, axis=1)

carac['holiday'] = parallelize_df(carac, par)

ValueError: Wrong number of items passed 132, placement implies 1

In [ ]:
carac.head()